<a href="https://colab.research.google.com/github/nhy1210/GoogleAnalyticDataProcessing/blob/main/ExtractData_FromRawSrc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import json
import ast 
import pickle 
import time
import datetime
from datetime import datetime, date
from apiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials


def extractData(individualID):

  ###########################################################
  # Get raw data from csv file
  filename = "Accenture_"+ individualID + ".csv"
  filepath = "/content/drive/My Drive/GA_RawData/Accenture/1_Jan2019_Sep2020/Part1/" + filename
  df_userID = pd.read_csv(filepath, dtype=str)

  #Loading session data from pandas frame df_userID
  session = df_userID["sessions"]

  ###########################################################
  #Declare a pandas frame for adding data
  pd_pathinfo = pd.DataFrame(columns=['UserID','sessionID','Act_datetime','Duration_Time','App_Time','Path', 'Title', 'Campaign']) # remove , 'Act_Time'
  #pd_userBehav = pd.DataFrame(columns=['UserID', 'Total_Act','Total_Time', 'Com_Act', 'Com_Time' ,'My_Act', 'My_Time', 'Reading_Act', 'Reading_Time','Off_Hour_Cnt', 'Off_Hour_Time','OutOf_Hour_Cnt', 'OutOf_Hour_Time'])
  pd_userBehav = pd.DataFrame(columns=['UserID', 'Total_Act', 'Com_Act', 'My_Act', 'Reading_Act','Other_Act', 'Off_Hour_Cnt', 'OutOf_Hour_Cnt', 
  'Total_Time','Com_Time' ,'My_Time', 'Reading_Time','Other_Time','Off_Hour_Time', 'OutOf_Hour_Time'])

  #12/11/2020
  pd_user = pd.DataFrame(columns=['Session', 'Avg_session_duration', 
                                  'Page_view', 'Avg_page_view', 
                                  'Avg_reading_page_view','Avg_learning_page_view_%', 
                                  'Avg_learning_duration', 'Avg_learning_duration_%', 
                                  'Duplicate_learning','No_of_unique_page_view'])

  ###########################################################
  #Iterate all the Session
  for iSeS in range (len(session)): 
    i_Session = session[iSeS]

    #Convert string to dictionary type 
    dict_session = ast.literal_eval(i_Session)

    '''
    #Example of dict_session
    {'sessionId': '1582167399', 'deviceCategory': 'desktop', 'platform': 'Windows', 'dataSource': 'web',
      'activities': [{'activityTime': '2020-02-20T13:56:39.796152Z', 'source': '(direct)', 'medium': '(none)', 'channelGrouping': 'Direct', 'campaign': '(not set)', 'keyword': '(not set)', 'hostname': 'accenture.myspringday.com.au', 'landingPagePath': '/articles/sleep-and-money', 'activityType': 'PAGEVIEW', 
      'pageview': {'pagePath': '/articles/sleep-and-money', 'pageTitle': 'Sleep and Money'}}], 
      'sessionDate': '2020-02-20'
    }
    '''
    #========================================================
    ses_id = dict_session["sessionId"]
    ind_activity= dict_session["activities"]

    #========================================================
    #Pre-condition time 
    campaign = ind_activity[0]["campaign"]
    #print(campaign)
    act_atime = ind_activity[0]["activityTime"]
    act_time = act_atime[11:-2]
    act_hm = act_time[:5]
    print ("Act_time : ", act_hm)
    time_prev = datetime.strptime(act_time, '%H:%M:%S.%f').time()
    time_end = time_prev 

    #========================================================
    for act in range (len(ind_activity)):
      act_atime = ind_activity[act]["activityTime"]
      act_date = act_atime[:10]
      act_time = act_atime[11:-2]

      #exchange time string to datatime type
      time_cur = datetime.strptime(act_time, '%H:%M:%S.%f').time()
      time_diff = datetime.combine(date.today(), time_prev) - datetime.combine(date.today(), time_cur)
      time_prev = time_cur
      if act == len(ind_activity) -1 : 
        time_end = time_cur
      
      time_diff_sec = time_diff.seconds
      #print("Time diff : ", time_diff_sec, type(time_diff_sec))
      #print(type(time_cur))
      #print("Current_Time : ", time_cur)
      #print("Prev_Time.   : ", time_prev)

      '''
      #calculate session time:
      time_ses_diff = datetime.combine(date.today(), time_end) - datetime.combine(date.today(), time_start)
      time_ses_dur =time_ses_diff.seconds 
      print() 
      '''
      app_time = int(act_time[:2])
      page_view = ind_activity[act]["pageview"]

      path = page_view["pagePath"]
      view = page_view["pageTitle"]
      #row = [individualID, ses_id, act_date,act_time,time_diff_sec, app_time, path, view,campaign] Separate date and time => error in tableau
      row = [individualID, ses_id, act_atime, time_diff_sec, app_time, path, view,campaign]
      pd_pathinfo.loc[len(pd_pathinfo)] = row
  
  '''
  #convert time string to datetime
  pd_pathinfo['Act_Time'] = pd.to_datetime(pd_pathinfo['Act_Time'])
  x = pd_pathinfo['Act_Time'][1] - pd_pathinfo['Act_Time'][2]
  print("Time diff: ", x)
  print(type(pd_pathinfo['Act_Time'][0]))
  '''



  ###########################################################
  #Add initial filter to pandas frame (#filter_keywords = ['Home','(not set)','Not found - Error 404'])
  filter_keywords = ['Not found - Error 404']
  pd_pathinfo_filter = pd_pathinfo[~pd_pathinfo.Title.isin(filter_keywords)]
  #print("Size of general information after filtering", pd_pathinfo_filter.shape)

  filename = "A_AllInfo_"+ individualID + ".csv"
  filepath = "/content/drive/My Drive/1.Testing_CourseINFO/" + filename
  pd_pathinfo_filter.to_csv(filepath)


  '''
  ###########################################################
  #Health Wellbeing Check
  #20200930 pntn
  pd_wct_filter = pd_pathinfo[pd_pathinfo['Title'].str.contains('Take the Wellbeing Check Tool', na=False)]
  #Save General User information to file #Have to consider again to calculate the duration in session
  filename = "A_WCT_"+ individualID + ".csv"
  filepath = "/content/drive/My Drive/DataCollection_FromRAW/Accenture/Jan19_Sep20_Part2/WCT/" + filename
  #20200930 pntn pd_pathinfo_filter.to_csv(filepath)
  if len(pd_wct_filter.index) > 0: 
    pd_wct_filter.to_csv(filepath)
  #20200110 total_act = len(pd_pathinfo.index)
  #20200110 sum_total = pd_pathinfo['Duration_Time'].sum()

  ###########################################################
  #Better Together
  pd_better = pd_pathinfo[pd_pathinfo['Title'].str.contains('Better Together', na=False)]
  #Save General User information to file #Have to consider again to calculate the duration in session
  filename = "A_BT_"+ individualID + ".csv"
  filepath = "/content/drive/My Drive/DataCollection_FromRAW/Accenture/Jan19_Sep20_Part2/Campaign/Better Together/" + filename
  #20200930 pntn pd_pathinfo_filter.to_csv(filepath)
  if len(pd_better.index) > 0: 
    pd_better.to_csv(filepath)


  ###########################################################
  #Keeping Connected
  pd_Kconnect = pd_pathinfo[pd_pathinfo['Title'].str.contains('Keeping Connected', na=False)]
  #Save General User information to file #Have to consider again to calculate the duration in session
  filename = "A_KC_"+ individualID + ".csv"
  filepath = "/content/drive/My Drive/DataCollection_FromRAW/Accenture/Jan19_Sep20_Part2/Campaign/Keeping Connected/" + filename
  #20200930 pntn pd_pathinfo_filter.to_csv(filepath)
  if len(pd_Kconnect.index) > 0: 
    pd_Kconnect.to_csv(filepath)

  ###########################################################
  #Link Your Heart
  pd_LuH = pd_pathinfo[pd_pathinfo['Title'].str.contains('Link Your Heart', na=False)]
  #Save General User information to file #Have to consider again to calculate the duration in session
  filename = "A_LuH_"+ individualID + ".csv"
  filepath = "/content/drive/My Drive/DataCollection_FromRAW/Accenture/Jan19_Sep20_Part2/Campaign/Link Your Heart/" + filename
  #20200930 pntn pd_pathinfo_filter.to_csv(filepath)
  if len(pd_LuH.index) > 0: 
    pd_LuH.to_csv(filepath)

  ###########################################################
  #Money Habits
  pd_MH = pd_pathinfo[pd_pathinfo['Title'].str.contains('Money Habits', na=False)]
  #Save General User information to file #Have to consider again to calculate the duration in session
  filename = "A_MH_"+ individualID + ".csv"
  filepath = "/content/drive/My Drive/DataCollection_FromRAW/Accenture/Jan19_Sep20_Part2/Campaign/Money Habits/" + filename
  #20200930 pntn pd_pathinfo_filter.to_csv(filepath)
  if len(pd_MH.index) > 0: 
    pd_MH.to_csv(filepath)

  ###########################################################
  #Step Up In September
  pd_StUp = pd_pathinfo[pd_pathinfo['Title'].str.contains('Step Up In September', na=False)]
  #Save General User information to file #Have to consider again to calculate the duration in session
  filename = "A_StUp_"+ individualID + ".csv"
  filepath = "/content/drive/My Drive/DataCollection_FromRAW/Accenture/Jan19_Sep20_Part2/Campaign/Step Up In September/" + filename
  #20200930 pntn pd_pathinfo_filter.to_csv(filepath)
  if len(pd_StUp.index) > 0: 
    pd_StUp.to_csv(filepath)


  ###########################################################
  #Sustainable Behaviour Change
  pd_Sus = pd_pathinfo[pd_pathinfo['Title'].str.contains('Sustainable Behaviour Change', na=False)]
  #Save General User information to file #Have to consider again to calculate the duration in session
  filename = "A_Sus_"+ individualID + ".csv"
  filepath = "/content/drive/My Drive/DataCollection_FromRAW/Accenture/Jan19_Sep20_Part2/Campaign/Others/Sustainable Behaviour Change/" + filename
  #20200930 pntn pd_pathinfo_filter.to_csv(filepath)
  if len(pd_Sus.index) > 0: 
    pd_Sus.to_csv(filepath)

  ###########################################################
  #Get ahead of your mental fitness
  pd_Men = pd_pathinfo[pd_pathinfo['Title'].str.contains('Get ahead of your mental fitness', na=False)]
  #Save General User information to file #Have to consider again to calculate the duration in session
  filename = "A_Men_"+ individualID + ".csv"
  filepath = "/content/drive/My Drive/DataCollection_FromRAW/Accenture/Jan19_Sep20_Part2/Campaign/Others/Get ahead of your mental fitness/" + filename
  #20200930 pntn pd_pathinfo_filter.to_csv(filepath)
  if len(pd_Men.index) > 0: 
    pd_Men.to_csv(filepath)


  ###########################################################
  #New Year, New You
  pd_NY = pd_pathinfo[pd_pathinfo['Title'].str.contains('New Year, New You', na=False)]
  #Save General User information to file #Have to consider again to calculate the duration in session
  filename = "A_NY_"+ individualID + ".csv"
  filepath = "/content/drive/My Drive/DataCollection_FromRAW/Accenture/Jan19_Sep20_Part2/Campaign/Others/New Year_New You/" + filename
  #20200930 pntn pd_pathinfo_filter.to_csv(filepath)
  if len(pd_NY.index) > 0: 
    pd_NY.to_csv(filepath)

  ###########################################################
  #got your back this May
  pd_May = pd_pathinfo[pd_pathinfo['Title'].str.contains('got your back this May', na=False)]
  #Save General User information to file #Have to consider again to calculate the duration in session
  filename = "A_May_"+ individualID + ".csv"
  filepath = "/content/drive/My Drive/DataCollection_FromRAW/Accenture/Jan19_Sep20_Part2/Campaign/Others/got your back this May/" + filename
  #20200930 pntn pd_pathinfo_filter.to_csv(filepath)
  if len(pd_May.index) > 0: 
    pd_May.to_csv(filepath)

  
  ###########################################################
  #Reading category  
  pd_reading_filter = pd_pathinfo[pd_pathinfo['Path'].str.contains('/articles/')]
  filename = "A_Rd_"+ individualID + ".csv"
  filepath = "/content/drive/My Drive/Extract_data_FromRAW/Accenture/Sep19_Feb20/Reading/" + filename
  pd_reading_filter.to_csv(filepath)
  read_act = len(pd_reading_filter.index)
  sum_read = pd_reading_filter['Duration_Time'].sum()
  #print("My reading: ",read_act )

  ###########################################################
  #Community category
  pd_com_filter = pd_pathinfo[pd_pathinfo['Path'].str.contains("/share-|/gallery", na=False)]
  filename = "A_Com_"+ individualID + ".csv"
  filepath = "/content/drive/My Drive/Extract_data_FromRAW/Accenture/Sep19_Feb20/Community/" + filename
  pd_com_filter.to_csv(filepath)
  com_act = len(pd_com_filter.index)
  sum_com = pd_com_filter['Duration_Time'].sum()
  #print("My community action: ",com_act )


  ###########################################################
  #My activity category
  pd_my_filter = pd_pathinfo[pd_pathinfo['Path'].str.contains('/my/')]
  filename = "A_Act_"+ individualID + ".csv"
  filepath = "/content/drive/My Drive/Extract_data_FromRAW/Accenture/Sep19_Feb20/My_Activity/" + filename
  pd_my_filter.to_csv(filepath)
  my_act = len(pd_my_filter.index)
  sum_myact = pd_my_filter['Duration_Time'].sum()


  ###########################################################
  #Office hours _ Access time //to find user behaviour

  pd_office_hour =  pd_pathinfo[(pd_pathinfo['App_Time'] >= 9) & (pd_pathinfo['App_Time'] <= 17)]
  #pd_office_hour.sort_values(by=['App_Time'], inplace=True)

  filename = "A_OH_"+ individualID + ".csv"
  filepath = "/content/drive/My Drive/Extract_data_FromRAW/Accenture/Sep19_Feb20/Office_Hours/" + filename
  pd_office_hour.to_csv(filepath)
  OH_cnt = len(pd_office_hour.index)
  sum_office_hour = pd_office_hour['Duration_Time'].sum()

  ###########################################################
  #Out of Office hours _ Access time //to find user behaviour
  outofOH_cnt = total_act - OH_cnt

  ###########################################################
  #pd_userBehavior dataframe
  ['UserID', 'Total_Act', 'Com_Act', 'My_Act', 'Reading_Act','Other_Act', 
  'Off_Hour_Cnt', 'OutOf_Hour_Cnt', 
  'Total_Time','Com_Time' ,'My_Time', 'Reading_Time','Other_Time',
  'Off_Hour_Time', 'OutOf_Hour_Time'])
  ###########################################################

  othe_act = total_act - (com_act + my_act + read_act)
  sum_other = sum_total - (sum_com  + sum_myact + sum_read)
  final_row = [individualID, total_act,  com_act, my_act, read_act , othe_act, 
               OH_cnt,  outofOH_cnt,
               sum_total, sum_com,sum_myact,sum_read, sum_other,
               sum_office_hour,sum_total - sum_office_hour]
  pd_userBehav.loc[len(pd_userBehav)]  = final_row
  #Save into file 
  filename = "UserBehav_"+ individualID + ".csv"
  filepath = "/content/drive/My Drive/Extract_data_FromRAW/Accenture/Sep19_Feb20/FinalCount/" + filename
  pd_userBehav.to_csv(filepath)
  '''
'''
SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
KEY_FILE_LOCATION = '/content/drive/My Drive/PythonGA/client_secrets.json'
#VIEW_ID = '180260411' #Success Apps
VIEW_ID = '179984294'
'''
def userId_data():
  #prepare UserID - loading from csv file to pandas frame
  df_userID = pd.read_csv('/content/drive/My Drive/PythonGA/User Explorer 20190101-20200930_P1.csv', dtype=str)
  #print(df_userID['Client Id'])
  return df_userID
'''
def initialize_analyticsreporting():
  """Initializes an Analytics Reporting API V4 service object.

  Returns:
    An authorized Analytics Reporting API V4 service object.
  """
  credentials = ServiceAccountCredentials.from_json_keyfile_name(
      KEY_FILE_LOCATION, SCOPES)

  # Build the service object.
  analytics = build('analyticsreporting', 'v4', credentials=credentials)

  return analytics


def get_report(analytics, IndUserID):
  """Queries the Analytics Reporting API V4.

  Args:
    analytics: An authorized Analytics Reporting API V4 service object.
  Returns:
    The Analytics Reporting API V4 response.
  """
  return analytics.userActivity().search(
        body = {
            "viewId": VIEW_ID,
            #"viewId" : "8925304082524788925",
            "user": {
                "type": "CLIENT_ID",
                #"userId": "1227477603.1582680286"
                "userId": IndUserID
            },
            "dateRange": {
                "startDate": "2019-09-01",
                "endDate": "2020-02-29",
            }
        },
        quotaUser='my-user-1'
    ).execute()

'''  

def main():

  #==========================================================
  #Loading UserID into pandas frame
  df_user = userId_data() 
  '''
  #==========================================================
  #Loading all UserIDS info from raw data to extract userful information
  for ind in df_user.index:
    clientID = df_user['Client Id'][ind]
    extractData(clientID)
    print("ClienID : ", clientID)

  '''
  #==========================================================
  # Loading one UserID info from raw data to extract userful information
  clientID = df_user['Client Id'][0]
  #clientID ="5422991.1569975384"
  #clientID = "1009165876.1580704555"
  extractData(clientID)
  print("ClienID : ", clientID)
  #==========================================================
  
  
  
if __name__ == '__main__':
  main()

Act_time :  00:53
Act_time :  22:16
ClienID :  1684510276.1546341413
